# Scaling_ML_Models.ipynb

## 簡介 (Introduction)

在大數據時代，單機運算不足以有效處理與訓練龐大的資料集與模型。**Apache Spark** 提供分散式計算能力，並透過 **MLlib** 提供機器學習的高階 API，在分散式環境中訓練、評估與調優 ML 模型。

本 Notebook 將示範：
- 使用 SparkSession 啟動分散式運算環境
- 讀取與準備資料
- 使用 Spark MLlib 建立機器學習 Pipeline（包括特徵工程與模型）
- 在 Spark 分散式環境中訓練模型（以邏輯回歸為例）
- 評估模型表現與交叉驗證
- 簡要展示參數調優（如透過 CrossValidator）

請確保您已安裝好 PySpark。
若未安裝，可透過：
```bash
!pip install pyspark
```

## 1. 初始化 SparkSession

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("Scaling_ML_Models") \
    .getOrCreate()

spark

## 2. 資料讀取與準備

此範例使用 Spark 自帶的範例資料（如 `sample_libsvm_data.txt`）做為示範。此檔案是 LIBSVM 格式，已包含標籤與特徵。

如需使用自己的資料，可使用 `spark.read.csv` 或其他方法讀取後自行特徵工程。

In [ ]:
# 載入 Spark 自帶範例資料
data = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")
data.show(10)
data.printSchema()

DataFrame 中有 `label` 與 `features` 欄位，`features` 是一個向量欄位。


## 3. 分割訓練集與測試集

In [ ]:
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)
print("Training Data Count:", train_data.count())
print("Test Data Count:", test_data.count())

## 4. 建立 ML Pipeline

Spark MLlib 的 Pipeline 可以將特徵處理與模型訓練串接。

此例中資料已經是 vectorized features，不需額外的特徵工程，
我們將直接使用 LogisticRegression。若有需要特徵工程，可使用 `VectorAssembler`, `StringIndexer`, `OneHotEncoder` 等 Transformer。
### 使用 LogisticRegression 模型

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

# 建立 Logistic Regression 實例
lr = LogisticRegression(maxIter=10, regParam=0.001)

# Pipeline: (此處無額外前處理，直接放入模型)
pipeline = Pipeline(stages=[lr])


## 5. 模型訓練

透過 `pipeline.fit()` 在 Spark 分散式環境中訓練模型。

In [ ]:
model = pipeline.fit(train_data)
print("Model trained.")

## 6. 模型預測與評估

使用訓練好的模型對測試集做預測，並計算基本的評估指標，如 Accuracy。

In [ ]:
predictions = model.transform(test_data)
predictions.select("label", "prediction", "probability").show(10)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Test Accuracy: {accuracy}")

## 7. 參數調優 (Parameter Tuning) 與 交叉驗證 (Cross-Validation)

可使用 `CrossValidator` 搭配 ParamGridBuilder 對模型的超參數進行調整並找到最佳參數組合。

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# 建立參數網格 (例如：調整 regParam )
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.001, 0.01, 0.1])
             .addGrid(lr.maxIter, [10, 50])
             .build())

# 使用 CrossValidator
cv = CrossValidator(estimator=pipeline,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator,
                    numFolds=3)

# 執行交叉驗證
cvModel = cv.fit(train_data)

bestModel = cvModel.bestModel
print("Best Model: ", bestModel.stages)


在分散式環境中，交叉驗證會平行執行不同的參數組合與資料折疊，以提高搜尋效率與速度。

驗證最佳模型的表現：

In [ ]:
cv_predictions = cvModel.transform(test_data)
cv_accuracy = evaluator.evaluate(cv_predictions)
print(f"CV Test Accuracy: {cv_accuracy}")

## 8. 結束 Spark Session

在完成後呼叫 `spark.stop()` 釋放資源。

In [ ]:
spark.stop()

## 總結

本 Notebook 展示了如何在分散式環境（透過 Spark）訓練機器學習模型，包括：
- 使用 SparkSession 啟動叢集運算環境
- 讀取 LIBSVM 格式資料
- 使用 Spark ML Pipeline 建立並訓練 Logistic Regression 模型
- 使用 DataFrame API 與 Evaluator 進行模型預測與評估
- 利用 CrossValidator 與 ParamGridBuilder 進行參數調優與交叉驗證

透過這些步驟，你可以在分散式環境中更有效率地訓練、評估並提升 ML 模型性能。